In [39]:
import utils
import pandas as pd
import numpy as np
from vs_feature_eng import feature_eng_v2

In [40]:
X, y = utils.get_train_data()

In [41]:
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import HistGradientBoostingRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

# DATA PREPROCESSING
# ==============================================================================
numeric_features = [### DATA
                    # Relative to dates
                    'year',
                    'season_sin',
                    'season_cos',
                    'month_sin',
                    'month_cos',
                    'weekday_sin',
                    'weekday_cos',
                    'hour_sin',
                    'hour_cos',
                    'is_weekend',
                    'is_holiday',
                    'is_covid',
                    # Relative to location:
                    'site_id',
                    'latitude',
                    'longitude',

                    ### EXTERNAL DATA
                    't', 
                    'etat_sol',
                    'cl',
                    "w2",
                    "nbas",
                    "nnuage1",
                    "n",
                    "raf10",
                    "ssfrai",
                    "rr12"
                    ]


preprocessor = ColumnTransformer(
    [('standard-scaler', StandardScaler(), numeric_features)],
    remainder='drop'
    )

# We create the full pipeline:
model = make_pipeline(
    feature_eng_v2.FeatureEngineering(),
    preprocessor,
    HistGradientBoostingRegressor(learning_rate=0.05, max_iter=300, max_depth=10) # default values
)

In [42]:
### TRAIN_TEST_split and RMSE measures:
def train_test_split_temporal(X, y, delta_threshold="30 days"):
    
    cutoff_date = X["date"].max() - pd.Timedelta(delta_threshold)
    mask = (X["date"] <= cutoff_date)
    X_train, X_valid = X.loc[mask], X.loc[~mask]
    y_train, y_valid = y[mask], y[~mask]

    return X_train, y_train, X_valid, y_valid

X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

In [43]:
model.fit(X_train, y_train)

Pipeline(steps=[('featureengineering',
                 <vs_feature_eng.feature_eng_v2.FeatureEngineering object at 0x12d21e5f0>),
                ('columntransformer',
                 ColumnTransformer(transformers=[('standard-scaler',
                                                  StandardScaler(),
                                                  ['year', 'season_sin',
                                                   'season_cos', 'month_sin',
                                                   'month_cos', 'weekday_sin',
                                                   'weekday_cos', 'hour_sin',
                                                   'hour_cos', 'is_weekend',
                                                   'is_holiday', 'is_covid',
                                                   'site_id', 'latitude',
                                                   'longitude', 't', 'etat_sol',
                                                   'cl', 'w2', 'nbas',
                                                   'nnuage1', 'n', 'raf10',
                                                   'ssfrai', 'rr12'])])),
                ('histgradientboostingregressor',
                 HistGradientBoostingRegressor(learning_rate=0.05, max_depth=10,
                                               max_iter=300))])

In [44]:
from sklearn.metrics import mean_squared_error

y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)

# Calculate and print RMSE for train and validation sets
print(f"Train set, RMSE={mean_squared_error(y_train, y_train_pred, squared=False):.2f}")
print(f"Validation set, RMSE={mean_squared_error(y_valid, y_valid_pred, squared=False):.2f}")

Train set, RMSE=0.58
Validation set, RMSE=0.55


/Applications/anaconda3/envs/bikes-count/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/Applications/anaconda3/envs/bikes-count/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [45]:
test_set = pd.read_parquet("./data/final_test.parquet")

In [46]:
predictions = model.predict(test_set)

In [47]:
# Prepare submission
output_df = pd.DataFrame({
    'Id': test_set.index,
    'log_bike_count': predictions
})

# Format log_bike_count:
output_df['log_bike_count'] = output_df['log_bike_count'].map(lambda x: f"{x:.4f}")

# Save to CSV:
output_df.to_csv('vs_sub_full_dataset_HGB_v2.csv', index=False)
print("Predictions saved to 'vs_sub_full_dataset_HGB_v2.csv'.")

Predictions saved to 'vs_sub_full_dataset_HGB_v2.csv'.


# __OPTUNA__

In [48]:
import optuna

In [49]:
def objective(trial):
    # Suggest hyperparameters
    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
    max_iter = trial.suggest_int('max_iter', 100, 500)
    max_depth = trial.suggest_int('max_depth', 3, 12)
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 1, 20)
    
    # Create a pipeline with the suggested hyperparameters
    model = make_pipeline(
        feature_eng_v2.FeatureEngineering(),
        preprocessor,
        HistGradientBoostingRegressor(
            learning_rate=learning_rate,
            max_iter=max_iter,
            max_depth=max_depth,
            min_samples_leaf=min_samples_leaf
        )
    )
    
    # Split your data into training and validation sets
    X_train, y_train, X_valid, y_valid = train_test_split_temporal(X, y)

    # Fit the model
    model.fit(X_train, y_train)
    
    # Make predictions and calculate the RMSE
    y_valid_pred = model.predict(X_valid)
    rmse = mean_squared_error(y_valid, y_valid_pred, squared=False)
    
    return rmse  # We want to minimize RMSE

# Run Optuna optimization
study = optuna.create_study(direction='minimize')  # Minimize the objective
study.optimize(objective, n_trials=100)  # Run for 100 trials

# Print the best parameters
print("Best trial:")
trial = study.best_trial

print(f"  RMSE: {trial.value}")
print("  Best hyperparameters:")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

# After optimization, you can use the best hyperparameters to train your final model
best_params = trial.params
final_model = make_pipeline(
    feature_eng_v2.FeatureEngineering(),
    preprocessor,
    HistGradientBoostingRegressor(
        learning_rate=best_params['learning_rate'],
        max_iter=best_params['max_iter'],
        max_depth=best_params['max_depth'],
        min_samples_leaf=best_params['min_samples_leaf']
    )
)

# Fit the final model on the full training set
final_model.fit(X_train, y_train)

# Make predictions on the validation set
y_valid_pred = final_model.predict(X_valid)

# Evaluate the model
print(f"Final validation RMSE: {mean_squared_error(y_valid, y_valid_pred, squared=False):.2f}")

[I 2024-12-10 16:21:19,031] A new study created in memory with name: no-name-b27909ff-68a8-44ea-832c-242c7183f4ca
/var/folders/kw/8v5sq5616c9bt_3ddzy97nq40000gn/T/ipykernel_4141/2420403170.py:3: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)
/Applications/anaconda3/envs/bikes-count/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
[I 2024-12-10 16:21:45,933] Trial 0 finished with value: 0.5611298013163405 and parameters: {'learning_rate': 0.03455044839379803, 'max_iter': 379, 'max_depth': 6, 'min_samples_leaf': 10}. Best is trial 0 with value: 0.5611298013163405.
/

Best trial:
  RMSE: 0.5426684010292198
  Best hyperparameters:
    learning_rate: 0.09443981884082536
    max_iter: 338
    max_depth: 6
    min_samples_leaf: 2
Final validation RMSE: 0.55


/Applications/anaconda3/envs/bikes-count/lib/python3.10/site-packages/sklearn/metrics/_regression.py:492: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [50]:
final_predictions = final_model.predict(test_set)

In [51]:
# Prepare submission
output_df = pd.DataFrame({
    'Id': test_set.index,
    'log_bike_count': final_predictions
})

# Format log_bike_count:
output_df['log_bike_count'] = output_df['log_bike_count'].map(lambda x: f"{x:.4f}")

# Save to CSV:
output_df.to_csv('vs_sub_full_dataset_HGB_vF.csv', index=False)
print("Predictions saved to 'vs_sub_full_dataset_HGB_vF.csv'.")

Predictions saved to 'vs_sub_full_dataset_HGB_vF.csv'.
